In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [76]:
data=pd.read_csv('../data/raw/BMW_Car_Sales_Classification.csv')
data.head()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low


In [77]:
numeric_cols=data.select_dtypes(include=np.number).columns
categorical_cols=data.select_dtypes(include='object').columns


In [78]:
numeric_cols

Index(['Year', 'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume'], dtype='object')

In [79]:
categorical_cols

Index(['Model', 'Region', 'Color', 'Fuel_Type', 'Transmission',
       'Sales_Classification'],
      dtype='object')

### Imputation Techniques

#### 1.Simple Imputation(Mean,Mode,Median)

In [80]:
imputer_mean=SimpleImputer(strategy='mean')
data[numeric_cols]=imputer_mean.fit_transform(data[numeric_cols])

imputer_mode=SimpleImputer(strategy='most_frequent')
data[categorical_cols]=imputer_mode.fit_transform(data[categorical_cols])

print('\n DataFrame after Simple imputation')
data=pd.DataFrame(data,columns=data.columns)
data.head()


 DataFrame after Simple imputation


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016.0,Asia,Red,Petrol,Manual,3.5,151748.0,98740.0,8300.0,High
1,i8,2013.0,North America,Red,Hybrid,Automatic,1.6,121671.0,79219.0,3428.0,Low
2,5 Series,2022.0,North America,Blue,Petrol,Automatic,4.5,10991.0,113265.0,6994.0,Low
3,X3,2024.0,Middle East,Blue,Petrol,Automatic,1.7,27255.0,60971.0,4047.0,Low
4,7 Series,2020.0,South America,Black,Diesel,Manual,2.1,122131.0,49898.0,3080.0,Low


#### 2. K-nearst Neighbors(KNN) imputation

In [81]:
imputer_knn=KNNImputer(n_neighbors=5)
data[numeric_cols]=imputer_knn.fit_transform(data[numeric_cols])
data=pd.DataFrame(data,columns=data.columns)
data.head()


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016.0,Asia,Red,Petrol,Manual,3.5,151748.0,98740.0,8300.0,High
1,i8,2013.0,North America,Red,Hybrid,Automatic,1.6,121671.0,79219.0,3428.0,Low
2,5 Series,2022.0,North America,Blue,Petrol,Automatic,4.5,10991.0,113265.0,6994.0,Low
3,X3,2024.0,Middle East,Blue,Petrol,Automatic,1.7,27255.0,60971.0,4047.0,Low
4,7 Series,2020.0,South America,Black,Diesel,Manual,2.1,122131.0,49898.0,3080.0,Low


#### 3. Multiple imputation by chained Equation(MICE)

In [82]:
imputer_mice=IterativeImputer(random_state=42)
data[numeric_cols]=imputer_mice.fit_transform(data[numeric_cols])
data=pd.DataFrame(data,columns=data.columns)
data.head()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016.0,Asia,Red,Petrol,Manual,3.5,151748.0,98740.0,8300.0,High
1,i8,2013.0,North America,Red,Hybrid,Automatic,1.6,121671.0,79219.0,3428.0,Low
2,5 Series,2022.0,North America,Blue,Petrol,Automatic,4.5,10991.0,113265.0,6994.0,Low
3,X3,2024.0,Middle East,Blue,Petrol,Automatic,1.7,27255.0,60971.0,4047.0,Low
4,7 Series,2020.0,South America,Black,Diesel,Manual,2.1,122131.0,49898.0,3080.0,Low


### Encoding Categorical Variables

In [83]:
for col in categorical_cols:
    print(data[col].value_counts())
    print('-------------------')

Model
7 Series    4666
i3          4618
i8          4606
3 Series    4595
5 Series    4592
X1          4570
X3          4497
X5          4487
M5          4478
X6          4478
M3          4413
Name: count, dtype: int64
-------------------
Region
Asia             8454
Middle East      8373
North America    8335
Europe           8334
Africa           8253
South America    8251
Name: count, dtype: int64
-------------------
Color
Red       8463
Silver    8350
Grey      8348
White     8304
Black     8273
Blue      8262
Name: count, dtype: int64
-------------------
Fuel_Type
Hybrid      12716
Petrol      12550
Electric    12471
Diesel      12263
Name: count, dtype: int64
-------------------
Transmission
Manual       25154
Automatic    24846
Name: count, dtype: int64
-------------------
Sales_Classification
Low     34754
High    15246
Name: count, dtype: int64
-------------------


#### One-Hot Encoding 
- one hot encoding -> [Transmission and Sales_Classification] 

In [84]:
encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore',drop='first')
encoded_features=encoder.fit_transform(data[['Transmission','Sales_Classification']])
encoded_df=pd.DataFrame(encoded_features,columns=['Transmission_ohe','Sales_Classification_ohe'])
encoded_df

,Transmission_ohe,Sales_Classification_ohe
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,1.0,1.0
...,...,...
49995,1.0,0.0
49996,1.0,0.0
49997,0.0,0.0
49998,0.0,0.0


In [85]:
data=pd.concat([data,encoded_df],axis=1)
data.drop(['Sales_Classification','Transmission'],axis=1,inplace=True)
data.head()

,Model,Year,Region,Color,Fuel_Type,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Transmission_ohe,Sales_Classification_ohe
0,5 Series,2016.0,Asia,Red,Petrol,3.5,151748.0,98740.0,8300.0,1.0,0.0
1,i8,2013.0,North America,Red,Hybrid,1.6,121671.0,79219.0,3428.0,0.0,1.0
2,5 Series,2022.0,North America,Blue,Petrol,4.5,10991.0,113265.0,6994.0,0.0,1.0
3,X3,2024.0,Middle East,Blue,Petrol,1.7,27255.0,60971.0,4047.0,0.0,1.0
4,7 Series,2020.0,South America,Black,Diesel,2.1,122131.0,49898.0,3080.0,1.0,1.0


#### Target Encoding for Model,Color,RegionFuel Type

In [88]:
model_encoding=data.groupby('Model')['Sales_Classification_ohe'].mean()
data['Model']=data['Model'].map(model_encoding)

In [89]:
color_encoding=data.groupby('Color')['Sales_Classification_ohe'].mean()
data['Color']=data['Color'].map(color_encoding)

In [91]:
region_encoding=data.groupby('Region')['Sales_Classification_ohe'].mean()
data['Region']=data['Region'].map(region_encoding)

In [93]:
fuel_type=data.groupby('Fuel_Type')['Sales_Classification_ohe'].mean()
data['Fuel_Type']=data['Fuel_Type'].map(fuel_type)

In [96]:
year=data.groupby('Year')['Sales_Classification_ohe'].mean()
data['Year']=data['Year'].map(year)

In [97]:
data.head()

,Model,Year,Region,Color,Fuel_Type,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Transmission_ohe,Sales_Classification_ohe
0,0.695775,0.702526,0.690324,0.701406,0.696016,3.5,151748.0,98740.0,8300.0,1.0,0.0
1,0.691055,0.698136,0.694301,0.701406,0.694951,1.6,121671.0,79219.0,3428.0,0.0,1.0
2,0.695775,0.677113,0.694301,0.690995,0.696016,4.5,10991.0,113265.0,6994.0,0.0,1.0
3,0.693573,0.683980,0.700585,0.690995,0.696016,1.7,27255.0,60971.0,4047.0,0.0,1.0
4,0.692027,0.691108,0.697612,0.698296,0.693550,2.1,122131.0,49898.0,3080.0,1.0,1.0


In [101]:
preprocessor=ColumnTransformer(
    transformers=[
        ('scale',StandardScaler(),['Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume'])
    ],
    remainder='passthrough'
).set_output(transform='pandas')

In [102]:
data=preprocessor.fit_transform(data)
data.head()

,scale__Engine_Size_L,scale__Mileage_KM,scale__Price_USD,scale__Sales_Volume,remainder__Model,remainder__Year,remainder__Region,remainder__Color,remainder__Fuel_Type,remainder__Transmission_ohe,remainder__Sales_Classification_ohe
0,0.250548,0.887814,0.911817,1.131530,0.695775,0.702526,0.690324,0.701406,0.696016,1.0,0.0
1,-1.632377,0.368717,0.160951,-0.573911,0.691055,0.698136,0.694301,0.701406,0.694951,0.0,1.0
2,1.241561,-1.541504,1.470514,0.674365,0.695775,0.677113,0.694301,0.690995,0.696016,0.0,1.0
3,-1.533276,-1.260805,-0.540950,-0.357231,0.693573,0.683980,0.700585,0.690995,0.696016,0.0,1.0
4,-1.136871,0.376656,-0.966867,-0.695729,0.692027,0.691108,0.697612,0.698296,0.693550,1.0,1.0


In [104]:
change_column_names={col:col.replace('scale__','').replace('remainder__','').replace('_ohe','') for col in data.columns}
change_column_names

{'scale__Engine_Size_L': 'Engine_Size_L',
 'scale__Mileage_KM': 'Mileage_KM',
 'scale__Price_USD': 'Price_USD',
 'scale__Sales_Volume': 'Sales_Volume',
 'remainder__Model': 'Model',
 'remainder__Year': 'Year',
 'remainder__Region': 'Region',
 'remainder__Color': 'Color',
 'remainder__Fuel_Type': 'Fuel_Type',
 'remainder__Transmission_ohe': 'Transmission',
 'remainder__Sales_Classification_ohe': 'Sales_Classification'}

In [106]:
data.rename(columns=change_column_names,inplace=True)
data.head()

,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Model,Year,Region,Color,Fuel_Type,Transmission,Sales_Classification
0,0.250548,0.887814,0.911817,1.131530,0.695775,0.702526,0.690324,0.701406,0.696016,1.0,0.0
1,-1.632377,0.368717,0.160951,-0.573911,0.691055,0.698136,0.694301,0.701406,0.694951,0.0,1.0
2,1.241561,-1.541504,1.470514,0.674365,0.695775,0.677113,0.694301,0.690995,0.696016,0.0,1.0
3,-1.533276,-1.260805,-0.540950,-0.357231,0.693573,0.683980,0.700585,0.690995,0.696016,0.0,1.0
4,-1.136871,0.376656,-0.966867,-0.695729,0.692027,0.691108,0.697612,0.698296,0.693550,1.0,1.0


In [108]:
data.to_csv('../data/processed/BMW Clean data.csv')